In [3]:
import os
import cv2
import numpy as np
import shutil

def my_bilateral_filter(image, d, sigma_color, sigma_space):
    """Apply a custom bilateral filter to the image."""
    image = image.astype(np.float32)
    H, W, C = image.shape
    half = d // 2

    padded = cv2.copyMakeBorder(image, half, half, half, half, cv2.BORDER_REFLECT)
    filtered = np.zeros_like(image)

    xs = np.arange(-half, half+1)
    ys = np.arange(-half, half+1)
    X, Y = np.meshgrid(xs, ys)
    spatial_kernel = np.exp(-((X**2 + Y**2) / (2 * sigma_space * sigma_space)))

    for i in range(H):
        for j in range(W):
            region = padded[i:i+d, j:j+d, :]
            center_pixel = padded[i+half, j+half, :]
            diff = region - center_pixel
            color_distance_sq = np.sum(diff * diff, axis=2)
            range_kernel = np.exp(-color_distance_sq / (2 * sigma_color * sigma_color))
            weights = spatial_kernel * range_kernel
            weights_sum = np.sum(weights)
            filtered[i, j, :] = np.sum(region * weights[:, :, None], axis=(0, 1)) / weights_sum

    return filtered.astype(np.uint8)

def preprocess_image(image_path, output_path, standard_size=(800, 600), d=9, sigma_color=75, sigma_space=75):
    """Preprocess the image and save it to the output path."""
    image = cv2.imread(image_path)
    if image is None:
        print(f"Warning: Unable to read image at {image_path}")
        return

    resized_image = cv2.resize(image, standard_size)
    lab_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2LAB)
    L, A, B = cv2.split(lab_image)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    L_clahe = clahe.apply(L)
    lab_clahe = cv2.merge((L_clahe, A, B))
    enhanced_bgr = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)
    preprocessed_image = my_bilateral_filter(enhanced_bgr, d, sigma_color, sigma_space)

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    cv2.imwrite(output_path, preprocessed_image)
    print(f"Processed image saved to {output_path}")

def preprocess_dataset(input_dir, output_dir):
    """Preprocess all images in the dataset and maintain folder structure."""
    if not os.path.exists(input_dir):
        print(f"Input directory {input_dir} does not exist.")
        return

    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                input_path = os.path.join(root, file)
                relative_path = os.path.relpath(input_path, input_dir)
                output_path = os.path.join(output_dir, relative_path)
                preprocess_image(input_path, output_path)

input_directory = 'smallerrefined'
output_directory = 'postprocessing_datasettttt_degree0'
preprocess_dataset(input_directory, output_directory)

Processed image saved to postprocessing_datasettttt_degree0\degree_1\img101_crop_1.jpg
Processed image saved to postprocessing_datasettttt_degree0\degree_1\img102_crop_0.jpg
Processed image saved to postprocessing_datasettttt_degree0\degree_1\img103_crop_0.jpg
Processed image saved to postprocessing_datasettttt_degree0\degree_1\img107_crop_0.jpg
Processed image saved to postprocessing_datasettttt_degree0\degree_1\img107_crop_1.jpg
Processed image saved to postprocessing_datasettttt_degree0\degree_1\img107_crop_2.jpg
Processed image saved to postprocessing_datasettttt_degree0\degree_1\img107_crop_3.jpg
Processed image saved to postprocessing_datasettttt_degree0\degree_1\img109_crop_0.jpg
Processed image saved to postprocessing_datasettttt_degree0\degree_1\img109_crop_1.jpg
Processed image saved to postprocessing_datasettttt_degree0\degree_1\img109_crop_2.jpg
Processed image saved to postprocessing_datasettttt_degree0\degree_1\img112_crop_0.jpg
Processed image saved to postprocessing_dat

KeyboardInterrupt: 